In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import itertools
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import pickle
%matplotlib inline

In [2]:
# Read in car and no car images
car_images = glob.glob('./image_set/vehicles/*.png')
no_car_images = glob.glob('./image_set/non-vehicles/*.png')
cars = []
notcars = []

for image in car_images:
    cars.append(image)
for image in no_car_images:
    notcars.append(image)

In [3]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [4]:
# Define a function to extract features from a list of images
def extract_features(imgs, cspace='BGR', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        # Using cv2 for consistency. it reads as BGR!!!
        image = cv2.imread(file)
        if cspace != 'BGR':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

In [5]:
# Let's do a systematic search of the parameters that affect the classifier.
n = 4
orient_grid = np.linspace(8, 12, n).astype(int)
cell_grid = np.linspace(1, 4, n).astype(int)
pix_grid = np.linspace(6, 12, n).astype(int)
colors = ['BGR', 'HSV', 'HLS', 'LUV', 'YUV', 'YCrCb']
chan_grid = [0, 1, 2, 'ALL']

In [6]:
def svm_car_classifier(cars, notcars, orient=9, pix_per_cell=8, cell_per_block=2,
                       hog_channel=0, colorspace='BGR', seed=np.random.randint(0, 100)):
    ''' Takes in a list of files representing car and no car images together with parametes for
    feature straction. Returns the fitted model and its score
    '''
    print('Using:',orient,'orientations',pix_per_cell, 'pixels per cell and',
          cell_per_block,'cells per block', 'in channel', hog_channel, 'with colorspace', colorspace)
    car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
    notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    # Split up data into randomized training and test sets
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=seed)
    # Use a linear SVC 
    svc = LinearSVC()
    svc.fit(X_train, y_train)
    # Check the score of the SVC
    score = svc.score(X_test, y_test)
    print('Test Accuracy of SVC = ', round(score, 4))
    return svc, score

In [7]:
# Search for the best color space
performance_color = []
models_color = []
orit = orient_grid[1]
pix = pix_grid[2]
cha = 'ALL'
cell = cell_grid[2]
for col in colors:
    # Call the classifier with a constant seed
    m, p = svm_car_classifier(cars, notcars, orient=orit, pix_per_cell=pix, cell_per_block=cell,
                       hog_channel=cha, colorspace=col, seed=3)
    models_color.append(m)
    performance_color.append(p)
    file_name = '-'.join((col, '_cell_{:d}'.format(cell),'pix{:d}'.format(pix),
                          'orient{:d}'.format(orit), 'channel'+str(cha)))
    file_name = '.'.join((file_name, 'pickle'))
    pickle.dump(m, open('./svm/' + file_name, 'wb'))

Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace BGR
Test Accuracy of SVC =  0.9533
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace HSV
Test Accuracy of SVC =  0.9794
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace HLS
Test Accuracy of SVC =  0.9794
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace LUV
Test Accuracy of SVC =  0.9811
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace YUV
Test Accuracy of SVC =  0.9834
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9848


In [8]:
# Search for the best cells per block
# We fix the color space to the best already found
col = sorted(zip(performance_color, colors))[-1][1]

performance_cell = []
models_cell = []
orit = orient_grid[1]
pix = pix_grid[2]
cha = 'ALL'
for cell in cell_grid:
    # Call the classifier with a constant seed
    m, p = svm_car_classifier(cars, notcars, orient=orit, pix_per_cell=pix, cell_per_block=cell,
                       hog_channel=cha, colorspace=col, seed=3)
    models_cell.append(m)
    performance_cell.append(p)
    file_name = '-'.join((col, '_cell_{:d}'.format(cell),'pix{:d}'.format(pix),
                          'orient{:d}'.format(orit), 'channel'+str(cha)))
    file_name = '.'.join((file_name, 'pickle'))
    pickle.dump(m, open('./svm/' + file_name, 'wb'))

Using: 9 orientations 10 pixels per cell and 1 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9758
Using: 9 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9851
Using: 9 orientations 10 pixels per cell and 3 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9848
Using: 9 orientations 10 pixels per cell and 4 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9806


In [9]:
# Search for the best number of orientations
# We also fix the cells per block already found
cell = sorted(zip(performance_cell, cell_grid))[-1][1]

performance_orient = []
models_orient = []
pix = pix_grid[2]
cha = 'ALL'
for orit in orient_grid:
    # Call the classifier with a constant seed
    m, p = svm_car_classifier(cars, notcars, orient=orit, pix_per_cell=pix, cell_per_block=cell,
                       hog_channel=cha, colorspace=col, seed=3)
    models_orient.append(m)
    performance_orient.append(p)
    file_name = '-'.join((col, '_cell_{:d}'.format(cell),'pix{:d}'.format(pix),
                          'orient{:d}'.format(orit), 'channel'+str(cha)))
    file_name = '.'.join((file_name, 'pickle'))
    pickle.dump(m, open('./svm/' + file_name, 'wb'))

Using: 8 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9851
Using: 9 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9848
Using: 10 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9868
Using: 12 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9868


In [13]:
# Search for the best number of pixels per cell
# We fix the orientations found
orit = sorted(zip(performance_orient, orient_grid))[-1][1]
# There is a tie so we use the one with less features
orit = 10

performance_pix = []
models_pix = []
cha = 'ALL'

for pix in pix_grid:
    # Call the classifier with a constant seed
    m, p = svm_car_classifier(cars, notcars, orient=orit, pix_per_cell=pix, cell_per_block=cell,
                       hog_channel=cha, colorspace=col, seed=3)
    models_pix.append(m)
    performance_pix.append(p)
    file_name = '-'.join((col, '_cell_{:d}'.format(cell),'pix{:d}'.format(pix),
                          'orient{:d}'.format(orit), 'channel'+str(cha)))
    file_name = '.'.join((file_name, 'pickle'))
    pickle.dump(m, open('./svm/' + file_name, 'wb'))

Using: 10 orientations 6 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9885
Using: 10 orientations 8 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9885
Using: 10 orientations 10 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9868
Using: 10 orientations 12 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9865


In [14]:
# Search for the best combination of channels
# We fix the number of pixels per cell found
pix = sorted(zip(performance_pix, pix_grid))[-1][1]

performance_cha = []
models_cha = []

for cha in chan_grid:
    # Call the classifier with a constant seed
    m, p = svm_car_classifier(cars, notcars, orient=orit, pix_per_cell=pix, cell_per_block=cell,
                       hog_channel=cha, colorspace=col, seed=3)
    models_cha.append(m)
    performance_cha.append(p)
    file_name = '-'.join((col, '_cell_{:d}'.format(cell),'pix{:d}'.format(pix),
                          'orient{:d}'.format(orit), 'channel'+str(cha)))
    file_name = '.'.join((file_name, 'pickle'))
    pickle.dump(m, open('./svm/' + file_name, 'wb'))

Using: 10 orientations 8 pixels per cell and 2 cells per block in channel 0 with colorspace YCrCb
Test Accuracy of SVC =  0.9482
Using: 10 orientations 8 pixels per cell and 2 cells per block in channel 1 with colorspace YCrCb
Test Accuracy of SVC =  0.9324
Using: 10 orientations 8 pixels per cell and 2 cells per block in channel 2 with colorspace YCrCb
Test Accuracy of SVC =  0.9136
Using: 10 orientations 8 pixels per cell and 2 cells per block in channel ALL with colorspace YCrCb
Test Accuracy of SVC =  0.9885


In [17]:
cha = sorted(zip(performance_cha, chan_grid))[-1][1]
performance = performance_cell + performance_cha + performance_color + performance_orient + performance_pix
print('Best performace', round(max(performance),3), 'is obtained using', orit, 'orientations',
      pix, 'pixels per cell', cell, 'cells per block in channel', cha, ' of colorspace', col)

Best performace 0.988 is obtained using 10 orientations 8 pixels per cell 2 cells per block in channel ALL  of colorspace YCrCb
